# Exploratory Data Analysis - Churn & Retention

This notebook performs comprehensive EDA on the cleaned churn features dataset.

## Overview
- Load data from DuckDB `analytics.churn_features`
- Analyze churn patterns by plan, region, usage trends, payment issues, and support load
- Generate visualizations and key insights

In [ ]:
# Imports
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Paths
PROJECT_ROOT = Path().resolve().parents[2]  # Go up from src/notebooks to project root
DB_PATH = PROJECT_ROOT / "churn.duckdb"
OUTPUT_DIR = PROJECT_ROOT / "eda_outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Database: {DB_PATH}")
print(f"Output directory: {OUTPUT_DIR}")

## 1. Load Data

Connect to DuckDB and load the cleaned churn features table.

In [ ]:
# Connect to DuckDB and load data
conn = duckdb.connect(DB_PATH.as_posix())
df = conn.execute("SELECT * FROM analytics.churn_features").df()
conn.close()

print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nColumns: {list(df.columns)}")

# Calculate churn statistics
churned = df['churn_label'] == 1
active = df['churn_label'] == 0
churn_rate = churned.sum() / (churned.sum() + active.sum()) * 100 if (churned.sum() + active.sum()) > 0 else 0
null_churn_pct = (df['churn_label'].isna().sum() / len(df)) * 100

print(f"\nChurn Statistics:")
print(f"  • Labeled customers: {df['churn_label'].notna().sum():,}")
print(f"  • Churn rate (labeled): {churn_rate:.2f}% ({churned.sum():,} churned / {active.sum():,} active)")
print(f"  • Null churn label: {null_churn_pct:.2f}% ({df['churn_label'].isna().sum():,} customers)")

df.head()

## 2. Overview Statistics

Summary statistics for numeric columns and key medians.

In [ ]:
# Get numeric columns (exclude customer_id)
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
numeric_cols = [col for col in numeric_cols if col != 'customer_id']

print("Numeric columns summary:")
df[numeric_cols].describe().round(2)

In [ ]:
# Key medians
print("Key Medians:")
print(f"  • Recency days: {df['recency_days'].median():.1f}")
print(f"  • Usage trend 30d: {df['usage_trend_30d'].median():.2f}")
print(f"  • Total payment issues: {df['total_payment_issues'].median():.0f}")
print(f"  • Failed payments (30d): {df['failed_payments_30d'].median():.0f}")
print(f"  • Total tickets: {df['total_tickets'].median():.0f}")
print(f"  • Active days: {df['active_days'].median():.0f}")
print(f"  • Avg sessions: {df['avg_sessions'].median():.2f}")
print(f"  • Avg usage minutes: {df['avg_usage_minutes'].median():.1f}")

## 3. Churn by Plan

Analyze churn rates across different subscription plans.

In [ ]:
# Calculate churn rate by plan
churn_by_plan = df.groupby('plan').agg({
    'churn_label': lambda x: ((x == 1).sum() / ((x == 1).sum() + (x == 0).sum()) * 100) if ((x == 1).sum() + (x == 0).sum()) > 0 else 0,
    'customer_id': 'count'
}).rename(columns={'churn_label': 'churn_rate', 'customer_id': 'total_customers'})
churn_by_plan = churn_by_plan.sort_values('churn_rate', ascending=False)

print("Churn rate by plan:")
churn_by_plan.round(2)

In [ ]:
# Plot churn by plan
fig = px.bar(
    churn_by_plan.reset_index(),
    x='plan',
    y='churn_rate',
    title='Churn Rate by Plan',
    labels={'churn_rate': 'Churn Rate (%)', 'plan': 'Plan'},
    text='churn_rate',
    color='churn_rate',
    color_continuous_scale='Reds'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    showlegend=False,
    height=500,
    xaxis={'categoryorder': 'total descending'}
)
fig.show()
fig.write_html(str(OUTPUT_DIR / "churn_by_plan.html"))

## 4. Churn by Region

Analyze churn rates across different geographic regions.

In [ ]:
# Calculate churn rate by region
churn_by_region = df.groupby('region').agg({
    'churn_label': lambda x: ((x == 1).sum() / ((x == 1).sum() + (x == 0).sum()) * 100) if ((x == 1).sum() + (x == 0).sum()) > 0 else 0,
    'customer_id': 'count'
}).rename(columns={'churn_label': 'churn_rate', 'customer_id': 'total_customers'})
churn_by_region = churn_by_region.sort_values('churn_rate', ascending=False)

print("Churn rate by region:")
churn_by_region.round(2)

In [ ]:
# Plot churn by region
fig = px.bar(
    churn_by_region.reset_index(),
    x='region',
    y='churn_rate',
    title='Churn Rate by Region',
    labels={'churn_rate': 'Churn Rate (%)', 'region': 'Region'},
    text='churn_rate',
    color='churn_rate',
    color_continuous_scale='Oranges'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    showlegend=False,
    height=500,
    xaxis={'categoryorder': 'total descending'}
)
fig.show()
fig.write_html(str(OUTPUT_DIR / "churn_by_region.html"))

## 5. Usage Trend vs Churn

Compare usage trend distributions between churned and active customers.

In [ ]:
# Filter to labeled customers only
df_labeled = df[df['churn_label'].notna()].copy()
df_labeled['churn_status'] = df_labeled['churn_label'].map({1: 'Churned', 0: 'Active'})

print(f"Comparing usage_trend_30d for {len(df_labeled):,} labeled customers...")
print(f"  • Active median: {df_labeled[df_labeled['churn_label'] == 0]['usage_trend_30d'].median():.2f}")
print(f"  • Churned median: {df_labeled[df_labeled['churn_label'] == 1]['usage_trend_30d'].median():.2f}")

In [ ]:
# Violin plot: Usage trend vs churn
plt.figure(figsize=(10, 6))
sns.violinplot(
    data=df_labeled,
    x='churn_status',
    y='usage_trend_30d',
    palette={'Active': 'green', 'Churned': 'red'},
    inner='box'
)
plt.title('Usage Trend 30d Distribution: Churned vs Active', fontsize=14, fontweight='bold')
plt.xlabel('Churn Status', fontsize=12)
plt.ylabel('Usage Trend 30d (minutes)', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "usage_trend_vs_churn.png", dpi=300, bbox_inches='tight')
plt.show()

## 6. Payment Issues vs Churn

Analyze relationship between payment issues and churn rate.

In [ ]:
# Bucket payment issues
df_labeled['payment_issues_bucket'] = pd.cut(
    df_labeled['total_payment_issues'],
    bins=[-1, 0, 1, 3, 5, 10, float('inf')],
    labels=['0', '1', '2-3', '4-5', '6-10', '11+']
)

churn_by_payment = df_labeled.groupby('payment_issues_bucket').agg({
    'churn_label': lambda x: ((x == 1).sum() / ((x == 1).sum() + (x == 0).sum()) * 100) if ((x == 1).sum() + (x == 0).sum()) > 0 else 0,
    'customer_id': 'count'
}).rename(columns={'churn_label': 'churn_rate', 'customer_id': 'total_customers'})

print("Churn rate by payment issues bucket:")
churn_by_payment.round(2)

In [ ]:
# Plot payment issues vs churn
fig = px.bar(
    churn_by_payment.reset_index(),
    x='payment_issues_bucket',
    y='churn_rate',
    title='Churn Rate by Payment Issues Count',
    labels={'churn_rate': 'Churn Rate (%)', 'payment_issues_bucket': 'Payment Issues Count'},
    text='churn_rate',
    color='churn_rate',
    color_continuous_scale='Reds'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    showlegend=False,
    height=500,
    xaxis={'categoryorder': 'category ascending'}
)
fig.show()
fig.write_html(str(OUTPUT_DIR / "payment_issues_vs_churn.html"))

## 7. Support Load vs Churn

Analyze relationship between support ticket count and churn rate.

In [ ]:
# Bucket total tickets
df_labeled['tickets_bucket'] = pd.cut(
    df_labeled['total_tickets'],
    bins=[-1, 0, 1, 2, 3, 5, 10, float('inf')],
    labels=['0', '1', '2', '3', '4-5', '6-10', '11+']
)

churn_by_tickets = df_labeled.groupby('tickets_bucket').agg({
    'churn_label': lambda x: ((x == 1).sum() / ((x == 1).sum() + (x == 0).sum()) * 100) if ((x == 1).sum() + (x == 0).sum()) > 0 else 0,
    'customer_id': 'count'
}).rename(columns={'churn_label': 'churn_rate', 'customer_id': 'total_customers'})

print("Churn rate by support tickets bucket:")
churn_by_tickets.round(2)

In [ ]:
# Plot support load vs churn
fig = px.bar(
    churn_by_tickets.reset_index(),
    x='tickets_bucket',
    y='churn_rate',
    title='Churn Rate by Support Ticket Count',
    labels={'churn_rate': 'Churn Rate (%)', 'tickets_bucket': 'Total Tickets'},
    text='churn_rate',
    color='churn_rate',
    color_continuous_scale='Blues'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    showlegend=False,
    height=500,
    xaxis={'categoryorder': 'category ascending'}
)
fig.show()
fig.write_html(str(OUTPUT_DIR / "support_load_vs_churn.html"))

## 8. Key Insights

Summary of key findings from the EDA.

In [ ]:
# Generate key insights
insights = []

# Insight 1: Plan churn
highest_plan_churn = churn_by_plan.index[0]
highest_plan_rate = churn_by_plan.loc[highest_plan_churn, 'churn_rate']
lowest_plan_churn = churn_by_plan.index[-1]
lowest_plan_rate = churn_by_plan.loc[lowest_plan_churn, 'churn_rate']
insights.append(f"• Plan '{highest_plan_churn}' has the highest churn rate ({highest_plan_rate:.1f}%), while '{lowest_plan_churn}' has the lowest ({lowest_plan_rate:.1f}%)")

# Insight 2: Region churn
highest_region_churn = churn_by_region.index[0]
highest_region_rate = churn_by_region.loc[highest_region_churn, 'churn_rate']
insights.append(f"• Region '{highest_region_churn}' shows the highest churn rate ({highest_region_rate:.1f}%)")

# Insight 3: Usage trend
active_trend = df_labeled[df_labeled['churn_label'] == 0]['usage_trend_30d'].median()
churned_trend = df_labeled[df_labeled['churn_label'] == 1]['usage_trend_30d'].median()
insights.append(f"• Churned customers show {churned_trend:.2f} min/day usage trend vs {active_trend:.2f} for active customers (median)")

# Insight 4: Payment issues
high_payment_churn = churn_by_payment.loc[churn_by_payment.index[-1], 'churn_rate']
low_payment_churn = churn_by_payment.loc[churn_by_payment.index[0], 'churn_rate']
insights.append(f"• Customers with 11+ payment issues have {high_payment_churn:.1f}% churn vs {low_payment_churn:.1f}% for zero issues")

# Insight 5: Support tickets
high_ticket_churn = churn_by_tickets.loc[churn_by_tickets.index[-1], 'churn_rate']
low_ticket_churn = churn_by_tickets.loc[churn_by_tickets.index[0], 'churn_rate']
insights.append(f"• Customers with 11+ support tickets have {high_ticket_churn:.1f}% churn vs {low_ticket_churn:.1f}% for zero tickets")

# Insight 6: Overall churn rate
insights.append(f"• Overall churn rate: {churn_rate:.2f}% ({churned.sum():,} churned / {active.sum():,} active)")

# Insight 7: Recency
high_recency = df_labeled[df_labeled['churn_label'] == 1]['recency_days'].median()
low_recency = df_labeled[df_labeled['churn_label'] == 0]['recency_days'].median()
insights.append(f"• Churned customers have median recency of {high_recency:.0f} days vs {low_recency:.0f} days for active customers")

print("KEY INSIGHTS:")
print("=" * 80)
for insight in insights:
    print(insight)
print("=" * 80)

## Summary

All plots have been saved to `eda_outputs/` directory:
- `churn_by_plan.html`
- `churn_by_region.html`
- `usage_trend_vs_churn.png`
- `payment_issues_vs_churn.html`
- `support_load_vs_churn.html`